In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score
import joblib

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/directional-forecasting-in-cryptocurrencies/sample_submission.csv
/kaggle/input/directional-forecasting-in-cryptocurrencies/train.csv
/kaggle/input/directional-forecasting-in-cryptocurrencies/test.csv


In [22]:
# Load the CSV file into a DataFrame

data = pd.read_csv('/kaggle/input/directional-forecasting-in-cryptocurrencies/train.csv')
data['timestamp'] = pd.to_datetime(data['timestamp'], unit='s')
data.set_index('timestamp', inplace=True)
data['number_of_trades'] = data['number_of_trades'].astype(int)
data['target'] = data['target'].astype(int)
data.head()

,open,high,low,close,volume,quote_asset_volume,number_of_trades,taker_buy_base_volume,taker_buy_quote_volume,target
timestamp,,,,,,,,,,
2018-05-04 22:01:00,0.90120,0.90130,0.90120,0.90130,134.98,121.646459,4,125.08,112.723589,1
2018-05-04 22:02:00,0.90185,0.90195,0.90185,0.90195,1070.54,965.505313,12,879.94,793.612703,0
2018-05-04 22:03:00,0.90140,0.90140,0.90139,0.90139,2293.06,2066.963991,5,0.00,0.000000,0
2018-05-04 22:04:00,0.90139,0.90140,0.90138,0.90139,6850.59,6175.000909,19,1786.30,1610.149485,0
2018-05-04 22:05:00,0.90139,0.90139,0.90130,0.90130,832.30,750.222624,3,784.82,707.428900,0


In [23]:
# Define predictors and target
predictors = ['open', 'high', 'low', 'close', 'volume', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_volume', 'taker_buy_quote_volume']
X = data[predictors]
y = data['target']

# Scale features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [24]:
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [25]:
# Initialize the Logistic Regression model
model = LogisticRegression()

# Train the model
model.fit(X_train, y_train)

LogisticRegression()

In [26]:
# Make predictions
y_pred = model.predict(X_test)

In [27]:
# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Precision
precision = precision_score(y_test, y_pred)
print(f'Precision: {precision}')

Accuracy: 0.5253387610486044
Precision: 0.5116135979080142


In [31]:
# Save the model
joblib.dump(model, 'logistic_regression_model.pkl')


['logistic_regression_model.pkl']

In [29]:
# Load the model
model = joblib.load('logistic_regression_model.pkl')

# Load the test data
test_data = pd.read_csv('/kaggle/input/directional-forecasting-in-cryptocurrencies/test.csv')

# Define predictors
X_test = test_data[predictors]

# Scale features
X_test_scaled = scaler.transform(X_test)

# Make predictions
y_pred = model.predict(X_test_scaled)

# Create a DataFrame with index and predictions
predictions_df = pd.DataFrame({'row_id': test_data.index, 'target': y_pred})

# Export predictions to CSV
predictions_df.to_csv('submission.csv', index=False)

print("Predictions saved to submission.csv")

Predictions saved to submission.csv
